In [ ]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

#'/content/drive/MyDrive/SEM6miniproject/dataset/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/dataset/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv'
#,'/content/drive/MyDrive/SEM6miniproject/dataset/Friday-WorkingHours-Morning.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/Monday-WorkingHours.pcap_ISCX.csv',
#'/content/drive/MyDrive/SEM6miniproject/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv'
#'/content/drive/MyDrive/SEM6miniproject/Tuesday-WorkingHours.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/Wednesday-workingHours.pcap_ISCX.csv']

files=['/content/drive/MyDrive/SEM6miniproject/dataset/Tuesday-WorkingHours.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/dataset/Wednesday-workingHours.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/dataset/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/dataset/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv']#,'/content/drive/MyDrive/SEM6miniproject/dataset/Friday-WorkingHours-Morning.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/Monday-WorkingHours.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/Tuesday-WorkingHours.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/Wednesday-workingHours.pcap_ISCX.csv']

In [ ]:
dfs=[]

for file in files:
    df_temp = pd.read_csv(file)
    dfs.append(df_temp)

df = pd.concat(dfs, ignore_index=True)

df.shape

(1597580, 79)

In [ ]:
# Ensure there are no infinite values
#df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop or fill rows with NaN after replacing infinity
#df = df.dropna()

#pd.set_option('display.max_rows', None)


#df.shape

In [ ]:
#scaling= df.drop([' Label'],axis=1)

# Encoding categorical columns (Label column)
#label_encoder = LabelEncoder()
#df[' Label'] = label_encoder.fit_transform(df[' Label'])

#print(df[' Label'])

# Scaling numerical features
#scaler = StandardScaler()
#scaled_features = scaler.fit_transform(scaling)


# Encoding categorical column 'Label'
#label_encoder = LabelEncoder()
#df[' Label'] = label_encoder.fit_transform(df[' Label'])

# Scaling numerical features (excluding 'Label')
#scaler = StandardScaler()
#scaled_features = scaler.fit_transform(df.drop([' Label'], axis=1))

# To see the shape after scaling:
#print(scaled_features.shape)


In [ ]:
# Creating DataFrame from scaled features
#df_scaled = pd.DataFrame(scaled_features, columns=df.drop([' Label'], axis=1).columns)
#df_scaled = pd.concat([df_scaled, df[' Label']], axis=1)
# Append the ' Label' column to the scaled features DataFrame

#df_scaled.info()

# Replace infinity values
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN values
df.dropna(inplace=True)

# Encode categorical column 'Label'
label_encoder = LabelEncoder()
df[' Label'] = label_encoder.fit_transform(df[' Label'])

# Fill remaining NaN values if necessary (before scaling)
df.fillna(df.mean(), inplace=True)

# Scaling numerical features (excluding 'Label')
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df.drop(columns=[' Label']))

# Creating DataFrame from scaled features
df_scaled = pd.DataFrame(scaled_features, columns=df.drop(columns=[' Label']).columns)

# Append the ' Label' column
df_scaled[' Label'] = df[' Label'].values

# Check for any remaining NaN values
print(df_scaled.isnull().sum().sum())  # Should be 0 if no missing values remain


0


In [ ]:
# Splitting dataset
X = df_scaled.drop(' Label', axis=1)
y = df_scaled[' Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y_train.isnull().sum()

0

In [ ]:
from sklearn.linear_model import LogisticRegression

# Initialize RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
# Fit the model
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# Make predictions
y_pred = rf_model.predict(X_test)

In [ ]:
# Accuracy & Classification Report
acc = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {acc:.4f}")
print("Classification Report:\n", report)


Accuracy: 0.9991
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    265592
           1       1.00      1.00      1.00      2078
           2       1.00      1.00      1.00     46051
           3       0.99      0.99      0.99      1135
           4       1.00      0.99      0.99      1147
           5       1.00      1.00      1.00      1569
           6       1.00      1.00      1.00         1
           7       1.00      0.80      0.89         5
           8       1.00      1.00      1.00      1142
           9       0.73      0.77      0.75       276
          10       1.00      0.25      0.40         4
          11       0.49      0.39      0.43       136

    accuracy                           1.00    319136
   macro avg       0.93      0.85      0.87    319136
weighted avg       1.00      1.00      1.00    319136



In [ ]:
y_pred

array([2, 0, 0, ..., 0, 0, 0])

In [ ]:
# Step 9: Inverse transform back to the original labels
y_pred = label_encoder.inverse_transform(y_pred)

# Step 10: Output the original predicted labels
print("Predicted Original Labels:", y_pred)

Predicted Original Labels: ['DoS Hulk' 'BENIGN' 'BENIGN' ... 'BENIGN' 'BENIGN' 'BENIGN']


In [ ]:
files2=['/content/drive/MyDrive/SEM6miniproject/dataset/Friday-WorkingHours-Morning.pcap_ISCX.csv','/content/drive/MyDrive/SEM6miniproject/dataset/Monday-WorkingHours.pcap_ISCX.csv']
dfs2=[]

for file2 in files2:
    df_temp2 = pd.read_csv(file2)
    dfs2.append(df_temp2)

df2 = pd.concat(dfs2, ignore_index=True)

In [ ]:
# Replace infinity values
df2.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN values
df2.dropna(inplace=True)

# Encode categorical column 'Label'
df2[' Label'] = label_encoder.fit_transform(df2[' Label'])

# Fill remaining NaN values if necessary (before scaling)
df2.fillna(df2.mean(), inplace=True)

# Scaling numerical features (excluding 'Label')
scaled_features2 = scaler.fit_transform(df2.drop(columns=[' Label']))

# Creating DataFrame from scaled features
df2_scaled = pd.DataFrame(scaled_features2, columns=df2.drop(columns=[' Label']).columns)

# Append the ' Label' column
df2_scaled[' Label'] = df2[' Label'].values

X2 = df2_scaled.drop(' Label', axis=1)
y2= df2_scaled[' Label']

In [ ]:
y2_pred = rf_model.predict(X2)

# Accuracy & Classification Report
acc = accuracy_score(y2, y2_pred)
report = classification_report(y2, y2_pred)

print(f"Accuracy: {acc:.4f}")
print("Classification Report:\n", report)

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9972
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    718436
           1       0.00      0.00      0.00      1956
           2       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0

    accuracy                           1.00    720392
   macro avg       0.25      0.25      0.25    720392
weighted avg       0.99      1.00      1.00    720392



In [ ]:
#import pickle
#with open('idsMODEL.pkl', 'wb') as file:
#    pickle.dump(model,file)

In [ ]:
df2[' Label'].unique()


array([0, 1])